In [1]:
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point



In [15]:
import osmnx as ox

print(f"OSMnx version: {ox.__version__}")

OSMnx version: 2.0.1


In [2]:
import osmnx as ox
import os

# Specify the place name (Oregon)
place_name = "Oregon, USA"

# Download the graph
graph = ox.graph_from_place(place_name, network_type="drive")  # You can change network_type

graph_proj = ox.project_graph(graph)

# Get the path to your Downloads folder
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")

# Create the full file path
file_path = os.path.join(downloads_path, "oregon_drive_network_projected.graphml")

# Save the graph as a GraphML file
ox.save_graphml(graph_proj, filepath=file_path)

print(f"Projected GraphML file saved to: {file_path}")

C:\Users\weckero\AppData\Roaming\Python\Python311\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 115 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


Projected GraphML file saved to: C:\Users\weckero\Downloads\oregon_drive_network_projected.graphml


In [4]:
import osmnx as ox
import geopandas as gpd

# Load the GraphML file
graph = ox.load_graphml(filepath= downloads_path + "/oregon_drive_network_projected.graphml") #replace with your file name.
graph_proj = ox.project_graph(graph)

In [8]:
#Example of loading point data.
basefolder = "C:/Users/weckero/Documents/Tobackup/"

points_gdf = gpd.read_file(basefolder + "people_points.shp") #replace with your file name.
#points_gdf = points_gdf.to_crs(graph_proj.graph_crs) #project the point data.

In [21]:
import osmnx as ox
import geopandas as gpd
import os
import numpy as np
import networkx as nx
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
import pandas as pd

# 1. Load the GraphML file
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")
documents_path = os.path.join(os.path.expanduser("~"), "Documents")
graph = ox.load_graphml(os.path.join(downloads_path, "oregon_drive_network_projected.graphml"))
graph_proj = ox.project_graph(graph)

# 2. Load Point Data
basefolder = "C:/Users/weckero/Documents/Tobackup/"
points_gdf = gpd.read_file(documents_path + "clipped_tester.shp")
points_gdf = points_gdf.to_crs(graph_proj.graph['crs'])

# 3. Find nearest nodes
# Check if nearest_nodes exists in the current version
if hasattr(ox, 'get_nearest_nodes'):
    nearest_nodes = ox.get_nearest_nodes(
        G=graph_proj,
        X=points_gdf.geometry.x,
        Y=points_gdf.geometry.y
    )
elif hasattr(ox.distance, 'nearest_nodes'):
    nearest_nodes = ox.distance.nearest_nodes(
        G=graph_proj,
        X=points_gdf.geometry.x,
        Y=points_gdf.geometry.y
    )
else:
    # Manual implementation as fallback
    nearest_nodes = []
    for idx, point in points_gdf.iterrows():
        nearest = None
        min_dist = float('inf')
        for node, data in graph_proj.nodes(data=True):
            dist = ((data['x'] - point.geometry.x) ** 2 + 
                    (data['y'] - point.geometry.y) ** 2) ** 0.5
            if dist < min_dist:
                nearest = node
                min_dist = dist
        nearest_nodes.append(nearest)
        print(f"Found nearest node {nearest} for point {idx+1}/{len(points_gdf)}")

# 4. Check if travel_time exists in the graph
sample_edge = list(graph_proj.edges(data=True))[0][2]
if 'travel_time' not in sample_edge:
    print("Adding travel time to graph edges...")
    if 'length' in sample_edge and 'speed_kph' in sample_edge:
        # If the graph has length and speed information
        for u, v, data in graph_proj.edges(data=True):
            data['travel_time'] = data['length'] / (data['speed_kph'] * 1000 / 3600)
    else:
        # If not, use default speed (30 km/h if not available)
        if hasattr(ox, 'add_edge_speeds'):
            graph_proj = ox.add_edge_speeds(graph_proj)
        else:
            for u, v, data in graph_proj.edges(data=True):
                data['speed_kph'] = 30  # Default speed
                
        if hasattr(ox, 'add_edge_travel_times'):
            graph_proj = ox.add_edge_travel_times(graph_proj)
        else:
            for u, v, data in graph_proj.edges(data=True):
                if 'length' in data and 'speed_kph' in data:
                    data['travel_time'] = data['length'] / (data['speed_kph'] * 1000 / 3600)
                else:
                    # Default 3 minutes per edge if we can't calculate
                    data['travel_time'] = 180

# 5. Generate Isochrones
travel_time = 3600  # 1 hour in seconds
isochrones_list = []

for i, node in enumerate(nearest_nodes):
    try:
        print(f"Processing node {i+1}/{len(nearest_nodes)}: {node}")
        
        # Calculate shortest paths using networkx directly - compatible with all versions
        try:
            # Get all nodes that can be reached within travel_time
            reachable_nodes = []
            # Use networkx's single_source_dijkstra_path_length
            lengths = nx.single_source_dijkstra_path_length(
                G=graph_proj, 
                source=node, 
                cutoff=travel_time, 
                weight='travel_time'
            )
            
            reachable_nodes = list(lengths.keys())
            
            if not reachable_nodes:
                print(f"No nodes within travel time for node {node}")
                continue
                
            # Get coordinates for reachable nodes
            node_points = []
            for n in reachable_nodes:
                try:
                    node_points.append((graph_proj.nodes[n]['x'], graph_proj.nodes[n]['y']))
                except KeyError:
                    # Skip nodes without coordinates
                    continue
            
            if len(node_points) < 3:
                print(f"Not enough points ({len(node_points)}) to create polygon for node {node}")
                continue
                
            # Create a convex hull for the isochrone
            try:
                from scipy.spatial import ConvexHull
                hull = ConvexHull(node_points)
                polygon_points = [node_points[i] for i in hull.vertices]
                
                # Create Polygon
                isochrone_polygon = Polygon(polygon_points)
                
                # If the polygon is valid, add it to our results
                if isochrone_polygon.is_valid:
                    # Copy attributes from the original point
                    point_info = {}
                    if i < len(points_gdf):
                        for col in points_gdf.columns:
                            if col != 'geometry':
                                point_info[col] = points_gdf.iloc[i][col]
                    
                    isochrone_gdf = gpd.GeoDataFrame(
                        {**point_info, 'node_id': str(node), 'travel_time_mins': travel_time/60},
                        geometry=[isochrone_polygon], 
                        crs=graph_proj.graph['crs']
                    )
                    
                    isochrones_list.append(isochrone_gdf)
                    print(f"Successfully created isochrone for node {node}")
                else:
                    print(f"Created invalid polygon for node {node}")
                
            except Exception as e:
                print(f"Error creating polygon for node {node}: {e}")
                    
        except Exception as e:
            print(f"Error calculating paths for node {node}: {e}")
            
    except Exception as e:
        print(f"Error processing node {node}: {e}")

# Combine all isochrones into a single GeoDataFrame
if isochrones_list:
    try:
        isochrones_gdf = pd.concat(isochrones_list)
        
        # 6. Save the isochrones
        isochrones_output = os.path.join(basefolder, "isochrones_points.shp")
        isochrones_gdf.to_file(isochrones_output)
        print(f"Saved {len(isochrones_list)} isochrones to {isochrones_output}")
    except Exception as e:
        print(f"Error saving isochrones: {e}")
        # Try to save individually
        for i, iso_gdf in enumerate(isochrones_list):
            try:
                iso_output = os.path.join(basefolder, f"isochrone_{i}.shp")
                iso_gdf.to_file(iso_output)
                print(f"Saved individual isochrone to {iso_output}")
            except Exception as e2:
                print(f"Could not save isochrone {i}: {e2}")
else:
    print("No isochrones were created successfully.")

Adding travel time to graph edges...
Processing node 1/4953: 36944611
Error creating polygon for node 36944611: If using all scalar values, you must pass an index
Processing node 2/4953: 12177744664
Error creating polygon for node 12177744664: If using all scalar values, you must pass an index
Processing node 3/4953: 4854467950
Error creating polygon for node 4854467950: If using all scalar values, you must pass an index
Processing node 4/4953: 7058457949
Error creating polygon for node 7058457949: If using all scalar values, you must pass an index
Processing node 5/4953: 7058457949
Error creating polygon for node 7058457949: If using all scalar values, you must pass an index
Processing node 6/4953: 7058457949
Error creating polygon for node 7058457949: If using all scalar values, you must pass an index
Processing node 7/4953: 4854467950
Error creating polygon for node 4854467950: If using all scalar values, you must pass an index
Processing node 8/4953: 36959155
Error creating polygon

In [22]:
import networkx as nx
import geopandas as gpd
import os
import numpy as np
from shapely.geometry import Polygon, Point
from scipy.spatial import ConvexHull
import pandas as pd

# 1. Load the GraphML file (still using osmnx just for loading)
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")
graph = nx.read_graphml(os.path.join(downloads_path, "oregon_drive_network_projected.graphml"))

# Extract CRS from the graph attributes (assuming it's stored there)
graph_crs = None
if 'crs' in graph.graph:
    graph_crs = graph.graph['crs']
else:
    # Default to a common projection for Oregon (UTM Zone 10N)
    graph_crs = "EPSG:32610"
    print(f"No CRS found in graph, using default: {graph_crs}")

# 2. Load Point Data
basefolder = "C:/Users/weckero/Documents/Tobackup/"
points_gdf = gpd.read_file(basefolder + "people_points.shp")
if graph_crs:
    points_gdf = points_gdf.to_crs(graph_crs)

# 3. Add travel time to edges if not present
print("Checking and adding travel time to edges...")
need_travel_time = False

# Check a sample edge
sample_edge = list(graph.edges(data=True))[0][2]
if 'travel_time' not in sample_edge:
    need_travel_time = True

if need_travel_time:
    # Add travel time based on length and speed (if available)
    for u, v, data in graph.edges(data=True):
        if 'length' in data and 'speed_kph' in data:
            # Convert km/h to m/s and calculate travel time
            speed_ms = float(data['speed_kph']) * 1000 / 3600
            data['travel_time'] = float(data['length']) / speed_ms
        else:
            # Default travel time of 3 minutes if we don't have speed/length
            data['travel_time'] = 180  # 3 minutes in seconds
    print("Added travel_time attribute to edges")

# 4. Find nearest nodes for each point
nearest_nodes = []
for idx, point in points_gdf.iterrows():
    # Get point coordinates
    point_x, point_y = point.geometry.x, point.geometry.y
    
    # Find the nearest node in the graph
    min_dist = float('inf')
    nearest = None
    
    for node, data in graph.nodes(data=True):
        # Skip nodes without coordinates
        if 'x' not in data or 'y' not in data:
            continue
            
        # Calculate Euclidean distance
        node_x, node_y = float(data['x']), float(data['y'])  # Convert to float to be safe
        dist = ((node_x - point_x) ** 2 + (node_y - point_y) ** 2) ** 0.5
        
        if dist < min_dist:
            nearest = node
            min_dist = dist
    
    if nearest:
        nearest_nodes.append(nearest)
        print(f"Found nearest node {nearest} for point {idx+1}/{len(points_gdf)}")
    else:
        print(f"WARNING: No nearest node found for point {idx+1}")
        nearest_nodes.append(None)  # Placeholder

# 5. Generate Isochrones
travel_time = 3600  # 1 hour in seconds
isochrones_list = []

for i, node in enumerate(nearest_nodes):
    if node is None:
        print(f"Skipping point {i+1} - no valid node found")
        continue
        
    print(f"Processing node {i+1}/{len(nearest_nodes)}: {node}")
    
    try:
        # Get all nodes reachable within travel_time using NetworkX
        lengths = nx.single_source_dijkstra_path_length(
            G=graph, 
            source=node, 
            cutoff=travel_time, 
            weight='travel_time'
        )
        
        reachable_nodes = list(lengths.keys())
        
        if not reachable_nodes:
            print(f"No nodes within travel time for node {node}")
            continue
            
        # Get coordinates for reachable nodes
        node_points = []
        for n in reachable_nodes:
            try:
                # Cast to float to avoid any string conversion issues
                node_points.append((float(graph.nodes[n]['x']), float(graph.nodes[n]['y'])))
            except (KeyError, ValueError):
                # Skip nodes without valid coordinates
                continue
        
        if len(node_points) < 3:
            print(f"Not enough points ({len(node_points)}) to create polygon for node {node}")
            continue
            
        # Create a convex hull for the isochrone
        try:
            hull = ConvexHull(node_points)
            polygon_points = [node_points[i] for i in hull.vertices]
            
            # Create Polygon
            isochrone_polygon = Polygon(polygon_points)
            
            # If the polygon is valid, add it to our results
            if isochrone_polygon.is_valid:
                # Copy attributes from the original point
                point_info = {}
                if i < len(points_gdf):
                    for col in points_gdf.columns:
                        if col != 'geometry':
                            val = points_gdf.iloc[i][col]
                            # Convert numpy types to Python types for shapefile compatibility
                            if isinstance(val, (np.integer, np.floating, np.bool_)):
                                val = val.item()
                            point_info[col] = val
                
                isochrone_gdf = gpd.GeoDataFrame(
                    {**point_info, 'node_id': str(node), 'travel_mins': float(travel_time/60)},
                    geometry=[isochrone_polygon], 
                    crs=graph_crs
                )
                
                isochrones_list.append(isochrone_gdf)
                print(f"Successfully created isochrone for node {node}")
            else:
                print(f"Created invalid polygon for node {node}")
            
        except Exception as e:
            print(f"Error creating polygon for node {node}: {e}")
                
    except Exception as e:
        print(f"Error calculating paths for node {node}: {e}")

# 6. Combine all isochrones into a single GeoDataFrame
if isochrones_list:
    try:
        isochrones_gdf = pd.concat(isochrones_list)
        
        # Save the isochrones
        isochrones_output = os.path.join(basefolder, "isochrones_points.shp")
        isochrones_gdf.to_file(isochrones_output)
        print(f"Saved {len(isochrones_list)} isochrones to {isochrones_output}")
    except Exception as e:
        print(f"Error combining/saving isochrones: {e}")
        # Try to save individually
        for i, iso_gdf in enumerate(isochrones_list):
            try:
                iso_output = os.path.join(basefolder, f"isochrone_{i}.shp")
                iso_gdf.to_file(iso_output)
                print(f"Saved individual isochrone to {iso_output}")
            except Exception as e2:
                print(f"Could not save isochrone {i}: {e2}")
else:
    print("No isochrones were created successfully.")

Checking and adding travel time to edges...
Added travel_time attribute to edges
Found nearest node 36944611 for point 1/4953
Found nearest node 12177744664 for point 2/4953
Found nearest node 4854467950 for point 3/4953
Found nearest node 7058457949 for point 4/4953
Found nearest node 7058457949 for point 5/4953
Found nearest node 7058457949 for point 6/4953
Found nearest node 4854467950 for point 7/4953
Found nearest node 36959155 for point 8/4953
Found nearest node 36959155 for point 9/4953
Found nearest node 36959155 for point 10/4953
Found nearest node 36959155 for point 11/4953
Found nearest node 36959155 for point 12/4953
Found nearest node 36959155 for point 13/4953
Found nearest node 36921184 for point 14/4953
Found nearest node 10181880066 for point 15/4953
Found nearest node 10181880066 for point 16/4953
Found nearest node 36936470 for point 17/4953
Found nearest node 36936470 for point 18/4953
Found nearest node 36936470 for point 19/4953
Found nearest node 36936470 for poi

In [6]:
import networkx as nx
import geopandas as gpd
import os
import numpy as np
from shapely.geometry import Polygon, Point
from scipy.spatial import ConvexHull
import pandas as pd

# 1. Load the GraphML file (using networkx directly)
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")
graph = nx.read_graphml(os.path.join(downloads_path, "oregon_drive_network_projected.graphml"))

# Extract CRS from the graph attributes
graph_crs = None
if 'crs' in graph.graph:
    graph_crs = graph.graph['crs']
else:
    # Default to a common projection for Oregon (UTM Zone 10N)
    graph_crs = "EPSG:32610"
    print(f"No CRS found in graph, using default: {graph_crs}")

# 2. Load Point Data
documents_path = os.path.join(os.path.expanduser("~"), "Documents")
basefolder = "C:/Users/weckero/Documents/"
points_gdf = gpd.read_file(basefolder + "clipped_tester.shp")
if graph_crs:
    points_gdf = points_gdf.to_crs(graph_crs)

# 3. Add travel time to edges if not present
print("Checking and adding travel time to edges...")
need_travel_time = False

# Check a sample edge
sample_edge = list(graph.edges(data=True))[0][2]
if 'travel_time' not in sample_edge:
    need_travel_time = True

if need_travel_time:
    # Add travel time based on length and speed (if available)
    for u, v, data in graph.edges(data=True):
        if 'length' in data and 'speed_kph' in data:
            # Convert km/h to m/s and calculate travel time
            speed_ms = float(data['speed_kph']) * 1000 / 3600
            data['travel_time'] = float(data['length']) / speed_ms
        else:
            # Default travel time of 3 minutes if we don't have speed/length
            data['travel_time'] = 600  # 10 minutes in seconds
    print("Added travel_time attribute to edges: default 10 minutes")

# 4. Find nearest nodes for each point
nearest_nodes = []
for idx, point in points_gdf.iterrows():
    # Get point coordinates
    point_x, point_y = point.geometry.x, point.geometry.y
    
    # Find the nearest node in the graph
    min_dist = float('inf')
    nearest = None
    
    for node, data in graph.nodes(data=True):
        # Skip nodes without coordinates
        if 'x' not in data or 'y' not in data:
            continue
            
        # Calculate Euclidean distance
        node_x, node_y = float(data['x']), float(data['y'])
        dist = ((node_x - point_x) ** 2 + (node_y - point_y) ** 2) ** 0.5
        
        if dist < min_dist:
            nearest = node
            min_dist = dist
    
    if nearest:
        nearest_nodes.append(nearest)
        print(f"Found nearest node {nearest} for point {idx+1}/{len(points_gdf)}")
    else:
        print(f"WARNING: No nearest node found for point {idx+1}")
        nearest_nodes.append(None)  # Placeholder

# 5. Generate Isochrones
travel_time = 600 #10 minutes
isochrones_list = []

for i, node in enumerate(nearest_nodes):
    if node is None:
        print(f"Skipping point {i+1} - no valid node found")
        continue
        
    print(f"Processing node {i+1}/{len(nearest_nodes)}: {node}")
    
    try:
        # Get all nodes reachable within travel_time using NetworkX
        lengths = nx.single_source_dijkstra_path_length(
            G=graph, 
            source=node, 
            cutoff=travel_time, 
            weight='travel_time'
        )
        
        reachable_nodes = list(lengths.keys())
        
        if not reachable_nodes:
            print(f"No nodes within travel time for node {node}")
            continue
            
        # Get coordinates for reachable nodes
        node_points = []
        for n in reachable_nodes:
            try:
                node_points.append((float(graph.nodes[n]['x']), float(graph.nodes[n]['y'])))
            except (KeyError, ValueError):
                # Skip nodes without valid coordinates
                continue
        
        if len(node_points) < 3:
            print(f"Not enough points ({len(node_points)}) to create polygon for node {node}")
            continue
            
        # Create a convex hull for the isochrone
        try:
            hull = ConvexHull(node_points)
            polygon_points = [node_points[i] for i in hull.vertices]
            
            # Create Polygon
            isochrone_polygon = Polygon(polygon_points)
            
            # If the polygon is valid, add it to our results
            if isochrone_polygon.is_valid:
                # Create a dictionary for the data row
                row_data = {'geometry': isochrone_polygon, 'node_id': str(node), 'travel_mins': float(travel_time/60)}
                
                # Add point attributes
                if i < len(points_gdf):
                    for col in points_gdf.columns:
                        if col != 'geometry':
                            val = points_gdf.iloc[i][col]
                            # Convert numpy types to Python types
                            if isinstance(val, (np.integer, np.floating, np.bool_)):
                                val = val.item()
                            row_data[col] = val
                
                # Create a DataFrame with a single row and convert to GeoDataFrame
                row_df = pd.DataFrame([row_data])
                isochrone_gdf = gpd.GeoDataFrame(
                    row_df, 
                    geometry='geometry',
                    crs=graph_crs
                )
                
                isochrones_list.append(isochrone_gdf)
                print(f"Successfully created isochrone for node {node}")
            else:
                print(f"Created invalid polygon for node {node}")
            
        except Exception as e:
            print(f"Error creating polygon for node {node}: {e}")
                
    except Exception as e:
        print(f"Error calculating paths for node {node}: {e}")

# 6. Combine all isochrones into a single GeoDataFrame
if isochrones_list:
    try:
        isochrones_gdf = pd.concat(isochrones_list)
        
        # Save the isochrones
        isochrones_output = os.path.join(basefolder, "isochrones_points.shp")
        isochrones_gdf.to_file(isochrones_output)
        print(f"Saved {len(isochrones_list)} isochrones to {isochrones_output}")
    except Exception as e:
        print(f"Error combining/saving isochrones: {e}")
        # Try to save individually
        for i, iso_gdf in enumerate(isochrones_list):
            try:
                iso_output = os.path.join(basefolder, f"isochrone_{i}.shp")
                iso_gdf.to_file(iso_output)
                print(f"Saved individual isochrone to {iso_output}")
            except Exception as e2:
                print(f"Could not save isochrone {i}: {e2}")
else:
    print("No isochrones were created successfully.")

Checking and adding travel time to edges...
Added travel_time attribute to edges: default 10 minutes
Found nearest node 36960892 for point 1/110
Found nearest node 36960892 for point 2/110
Found nearest node 36982363 for point 3/110
Found nearest node 36982363 for point 4/110
Found nearest node 36982363 for point 5/110
Found nearest node 36960892 for point 6/110
Found nearest node 36960892 for point 7/110
Found nearest node 36960892 for point 8/110
Found nearest node 36960892 for point 9/110
Found nearest node 36960892 for point 10/110
Found nearest node 36960892 for point 11/110
Found nearest node 36960892 for point 12/110
Found nearest node 36960897 for point 13/110
Found nearest node 36960897 for point 14/110
Found nearest node 36926678 for point 15/110
Found nearest node 36956706 for point 16/110
Found nearest node 36926678 for point 17/110
Found nearest node 36956706 for point 18/110
Found nearest node 36956706 for point 19/110
Found nearest node 36956706 for point 20/110
Found ne

C:\Users\weckero\AppData\Local\Temp\ipykernel_18256\1118106271.py:168: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  isochrones_gdf.to_file(isochrones_output)
C:\Users\weckero\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'travel_mins' to 'travel_min'
  ogr_write(


In [7]:
import osmnx as ox
import geopandas as gpd
import os

# 1. Load the GraphML file (using OSMnx)
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")
graph = ox.load_graphml(os.path.join(downloads_path, "oregon_drive_network_projected.graphml"))
graph_proj = ox.project_graph(graph)

# 2. Load Point Data
basefolder = "C:/Users/weckero/Documents/"
points_gdf = gpd.read_file(basefolder + "clipped_tester.shp")
points_gdf = points_gdf.to_crs(graph_proj.graph['crs'])

# 3. Find nearest nodes
nearest_nodes = ox.distance.nearest_nodes(
    G=graph_proj,
    X=points_gdf.geometry.x,
    Y=points_gdf.geometry.y,
)

# 4. Generate Isochrones
travel_time = 600  # 10 minutes
isochrones_list = []

for i, node in enumerate(nearest_nodes):
    try:
        isochrone_polygon = ox.get_polygon(
            G=graph_proj,
            polygon=ox.distance.extended_isochrone_polygons(
                G=graph_proj,
                center_node=node,
                dist=travel_time,
                edge_buff=2.5,
                node_buff=0.5,
                circ_dist=1000,
                num_circles=6,
            ),
        )
        isochrone_gdf = gpd.GeoDataFrame(
            geometry=[isochrone_polygon], crs=graph_proj.graph['crs']
        )

        # Add point attributes
        if i < len(points_gdf):
            for col in points_gdf.columns:
                if col != 'geometry':
                    isochrone_gdf[col] = points_gdf.iloc[i][col]

        isochrones_list.append(isochrone_gdf)
        print(f"Successfully created isochrone for node {node}")

    except Exception as e:
        print(f"Error creating isochrone for node {node}: {e}")

# 5. Combine all isochrones into a single GeoDataFrame
if isochrones_list:
    try:
        isochrones_gdf = gpd.pd.concat(isochrones_list)
        isochrones_output = os.path.join(basefolder, "isochrones_points2.shp")
        isochrones_gdf.to_file(isochrones_output)
        print(f"Saved {len(isochrones_list)} isochrones to {isochrones_output}")
    except Exception as e:
        print(f"Error combining/saving isochrones: {e}")
else:
    print("No isochrones were created successfully.")

Error creating isochrone for node 36960892: module 'osmnx' has no attribute 'get_polygon'
Error creating isochrone for node 36960892: module 'osmnx' has no attribute 'get_polygon'
Error creating isochrone for node 36982363: module 'osmnx' has no attribute 'get_polygon'
Error creating isochrone for node 36982363: module 'osmnx' has no attribute 'get_polygon'
Error creating isochrone for node 36982363: module 'osmnx' has no attribute 'get_polygon'
Error creating isochrone for node 36960892: module 'osmnx' has no attribute 'get_polygon'
Error creating isochrone for node 36960892: module 'osmnx' has no attribute 'get_polygon'
Error creating isochrone for node 36960892: module 'osmnx' has no attribute 'get_polygon'
Error creating isochrone for node 36960892: module 'osmnx' has no attribute 'get_polygon'
Error creating isochrone for node 36960892: module 'osmnx' has no attribute 'get_polygon'
Error creating isochrone for node 36960892: module 'osmnx' has no attribute 'get_polygon'
Error crea